In [9]:
# vacunados region edad
import pandas as pd
src_path = "local_postprocessed_data/vacunas_diarias_edad_sexo.csv"

df_vacunas = pd.read_csv(src_path)
df_vacunas

,Dosis,datetime,60 o más,Menores de 60,60 o más (%),Menores de 60 (%)
0,Primera,2021-02-21 10:39:04 -0300,1906756.0,926040.0,0.573869,0.058634
1,Primera,2021-02-21 23:00:26 -0300,1908632.0,927233.0,0.574434,0.058710
2,Primera,2021-02-22 16:48:09 +0000,1931934.0,937102.0,0.581447,0.059334
3,Primera,2021-02-22 23:55:53 +0000,1972755.0,966058.0,0.593732,0.061168
4,Primera,2021-02-23 16:34:39 +0000,1990587.0,984316.0,0.599099,0.062324
...,...,...,...,...,...,...
63,Segunda,2021-03-10 10:36:32 +0000,594967.0,413801.0,0.179065,0.026201
64,Segunda,2021-03-10 15:28:39 +0000,673225.0,437439.0,0.202618,0.027697
65,Segunda,2021-03-10 23:06:29 +0000,788378.0,481873.0,0.237275,0.030511
66,Segunda,2021-03-11 17:56:01 +0000,859156.0,507693.0,0.258577,0.032146


In [178]:
import subprocess

file_path = 'Datos-COVID19/output/producto78/total_vacunados_sexo_edad.csv'



def get_historic_file_git(file_path):

    filename = file_path.split('/')[-1]
    file_path_dir = file_path.replace(filename, '')

    subprocess.run(['git', 'checkout','master'], cwd=file_path_dir)

    subprocess.run(f'git log --pretty=format:"%h|%ad| %s %d [%an]" --date=iso {filename} > commits.txt', shell=True, cwd=file_path_dir)

    commits_df = pd.read_csv(f'{file_path_dir}/commits.txt', sep='|', names=['commit','datetime','comment'])

    # solo para obtener el header
    df = pd.read_csv(file_path, nrows=0)

    for commit_row in commits_df.iterrows():
        subprocess.run(['git', 'checkout',commit_row[1]['commit']], cwd=file_path_dir)
        
        aux = pd.read_csv(file_path)
        aux['datetime'] = commit_row[1]['datetime']
        
        df = pd.concat([df, aux])

    return df


df_vacunas = get_historic_file_git(file_path)


In [180]:
df_vacunas = df_vacunas[pd.notna(df_vacunas['datetime'])]
df_grouped = df_vacunas.groupby(['Dosis','datetime']).sum()

over_60_cols = set(filter(lambda x: int(x)>=60, df_grouped.columns))
under_60_cols = set(df_grouped.columns).difference(over_60_cols)

df_grouped['60 o mas'] = df_grouped[over_60_cols].sum(axis=1)
df_grouped['Menores de 60'] = df_grouped[under_60_cols].sum(axis=1)

df_grouped = df_grouped[['60 o mas','Menores de 60']]

In [181]:
demo = pd.read_csv('local_postprocessed_data/Chile-2020.csv')

over_60_tot = demo.iloc[12:][['M','F']].sum().sum()
under_60_tot = demo.iloc[0:12][['M','F']].sum().sum()

df_grouped['60 o mas (%)'] = df_grouped['60 o mas']/over_60_tot
df_grouped['Menores de 60 (%)'] = df_grouped['Menores de 60']/under_60_tot

In [182]:
df_grouped.to_csv('local_postprocessed_data/vacunas_diarias_edad_sexo.csv')

In [183]:
# casos genero-edad
src_path = "Datos-COVID19/output/producto16/CasosGeneroEtarioEtapaClinica.csv"
df_casos = pd.read_csv(src_path)

df_casos['Etapa clinica'] = df_casos['Etapa clinica'].fillna('Ninguna')
df_casos = df_casos[df_casos['Etapa clinica']=='Ninguna'].groupby('Grupo de edad').sum().transpose()


In [164]:
df_casos_diarios = df_casos.diff(1)

In [166]:
df_casos_diarios.drop(df_casos_diarios.tail(1).index, inplace=True)

In [168]:
over_60_cols = set(['60 - 64 años','65 - 69 años','70 - 74 años','75 - 79 años','80 y más años'])
under_60_cols = set(df_casos_diarios.columns).difference(over_60_cols)


In [169]:
df_casos_diarios['60 o mas'] = df_casos_diarios[over_60_cols].sum(axis=1)
df_casos_diarios['Menores de 60'] = df_casos_diarios[under_60_cols].sum(axis=1)

In [170]:
df_casos_diarios = df_casos_diarios.reset_index().rename(columns={'index':'date'})

In [171]:
df_casos_diarios

Grupo de edad,date,00 - 04 años,05 - 09 años,10 - 14 años,15 - 19 años,20 - 24 años,25 - 29 años,30 - 34 años,35 - 39 años,40 - 44 años,45 - 49 años,50 - 54 años,55 - 59 años,60 - 64 años,65 - 69 años,70 - 74 años,75 - 79 años,80 y más años,60 o mas,Menores de 60
0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,2020-03-26,2.0,1.0,2.0,3.0,14.0,26.0,36.0,30.0,28.0,20.0,30.0,13.0,15.0,7.0,9.0,1.0,3.0,35.0,205.0
2,2020-03-27,0.0,0.0,1.0,6.0,7.0,26.0,24.0,18.0,19.0,23.0,16.0,12.0,9.0,12.0,4.0,4.0,1.0,30.0,152.0
3,2020-03-28,2.0,3.0,4.0,4.0,17.0,31.0,40.0,35.0,20.0,27.0,32.0,29.0,17.0,9.0,6.0,5.0,8.0,45.0,244.0
4,2020-03-29,1.0,3.0,0.0,4.0,12.0,16.0,15.0,23.0,28.0,22.0,6.0,15.0,16.0,11.0,5.0,3.0,3.0,38.0,145.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2021-02-01,263.0,331.0,464.0,736.0,1308.0,1485.0,1443.0,1221.0,1081.0,1163.0,1055.0,977.0,750.0,484.0,397.0,263.0,355.0,2249.0,11527.0
115,2021-02-05,302.0,324.0,446.0,871.0,1351.0,1710.0,1728.0,1429.0,1237.0,1250.0,1175.0,1148.0,854.0,621.0,464.0,324.0,440.0,2703.0,12971.0
116,2021-02-08,221.0,282.0,408.0,669.0,1028.0,1389.0,1364.0,1112.0,968.0,985.0,916.0,907.0,711.0,500.0,348.0,254.0,313.0,2126.0,10249.0
117,2021-02-12,293.0,292.0,447.0,834.0,1382.0,1671.0,1742.0,1381.0,1212.0,1234.0,1192.0,1099.0,818.0,598.0,409.0,295.0,400.0,2520.0,12779.0


In [172]:
df_casos_diarios['Totales'] = df_casos_diarios[['60 o mas','Menores de 60']].sum(axis=1)
df_casos_diarios = df_casos_diarios[['date','60 o mas','Menores de 60','Totales']]

In [173]:
df_casos_diarios.to_csv('local_postprocessed_data/casos_diarios.csv')

In [69]:
# UCI
src_path = "Datos-COVID19/output/producto9/HospitalizadosUCIEtario_T.csv"

df_UCI = pd.read_csv(src_path)
df_UCI.rename(columns={'Grupo de edad':'date'}, inplace=True)
df_UCI['60 o mas'] = df_UCI[['60-69','>=70']].sum(axis=1)
df_UCI['Menores de 60'] = df_UCI[['<=39','40-49','50-59']].sum(axis=1)
df_UCI['Totales'] = df_UCI[['Menores de 60', '60 o mas']].sum(axis=1)
df_UCI.to_csv('local_postprocessed_data/uci_diarios.csv')

In [114]:
# camas UCI

src_path = "Datos-COVID19/output/producto58/Camas_UCI_diarias_t.csv"
camas_uci = pd.read_csv(src_path)

In [115]:
camas_uci_proc = camas_uci[['Region','Total','Total.2']].rename(columns={'Region':'date','Total':'Camas UCI Habilitadas', 'Total.2':'Camas no Covid-19 ocupadas'})

camas_uci_proc.drop(camas_uci_proc.tail(1).index, inplace=True)

In [116]:
camas_uci_proc.to_csv('local_postprocessed_data/camas_uci.csv')